In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.7 MB/s eta 0:00:00


In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import create_optimizer
from transformers import TFAutoModelForSequenceClassification
from transformers.keras_callbacks import KerasMetricCallback
from transformers import AutoModel
from transformers import pipeline
import datasets
import evaluate

In [4]:
data = pd.read_csv("/content/drive/MyDrive/CSDF Mini Project/spam_dataset.csv")
data.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5171 non-null   int64 
 1   label       5171 non-null   object
 2   text        5171 non-null   object
 3   label_num   5171 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 161.7+ KB


In [6]:
data.drop(columns=['Unnamed: 0', 'label'], inplace=True)

In [7]:
data.columns = ['text', 'label']

In [8]:
data.head()

,text,label
0,Subject: enron methanol ; meter # : 988291\r\n...,0
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs\r\nthis deal is t...,0


In [9]:
X = data['text']
y = data['label']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3619,), (1552,), (3619,), (1552,))

In [12]:
model_checkpoint = 'albert-base-v2'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [13]:
def preprocess_function(examples):
    return tokenizer(examples, truncation=True)

In [14]:
X_train_tokenized = preprocess_function(X_train.to_list())
X_test_tokenized = preprocess_function(X_test.to_list())

In [15]:
tokenized_train_data = {}
tokenized_train_data['input_ids'] = X_train_tokenized['input_ids']
tokenized_train_data['attention_mask'] = X_train_tokenized['attention_mask']
tokenized_train_data['labels'] = y_train

tokenized_test_data = {}
tokenized_test_data['input_ids'] = X_test_tokenized['input_ids']
tokenized_test_data['attention_mask'] = X_test_tokenized['attention_mask']
tokenized_test_data['labels'] = y_test

In [16]:
train_data_df = pd.DataFrame(data=dict(tokenized_train_data))
test_data_df = pd.DataFrame(data=dict(tokenized_test_data))

In [17]:
tokenized_train_data = datasets.Dataset.from_pandas(train_data_df)
tokenized_test_data = datasets.Dataset.from_pandas(test_data_df)

In [46]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [19]:
id2label = {0: "Not Spam", 1: "Spam"}
label2id = {"Not Spam": 0, "Spam": 1}

In [20]:
accuracy = evaluate.load("accuracy")

In [21]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [22]:
batch_size = 16
num_epochs = 5
batches_per_epoch = len(X_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [23]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id
)

All PyTorch model weights were used when initializing TFAlbertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFAlbertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_train_data,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)
tf_validation_set = model.prepare_tf_dataset(
    tokenized_test_data,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [26]:
model.compile(optimizer=optimizer, metrics=['accuracy'])

In [27]:
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [28]:
model.fit(tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=[metric_callback])

Epoch 1/3
226/226 [==============================] - 578s 2s/step - loss: 0.1608 - accuracy: 0.9800 - val_loss: 0.0555 - val_accuracy: 0.9800
Epoch 2/3
226/226 [==============================] - 568s 3s/step - loss: 0.0512 - accuracy: 0.9710 - val_loss: 0.0945 - val_accuracy: 0.9710
Epoch 3/3
226/226 [==============================] - 565s 3s/step - loss: 0.0136 - accuracy: 0.9858 - val_loss: 0.0487 - val_accuracy: 0.9858


In [29]:
model.evaluate(tf_validation_set)

97/97 [==============================] - 69s 714ms/step - loss: 0.0487 - accuracy: 0.9858


[0.04866095632314682, 0.9858247637748718]

In [30]:
from huggingface_hub import notebook_login

In [31]:
notebook_login()

In [32]:
model.save('albert_spam_filter.keras')

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [33]:
!cp "/content/albert_spam_filter.keras" "/content/drive/MyDrive/CSDF Mini Project/"

In [34]:
model.push_to_hub("NotShrirang/albert-spam-filter")

tf_model.h5:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Model is available at https://huggingface.co/NotShrirang/albert-spam-filter

In [60]:
test_email = """director CRD <directorcrd@mmcoe.edu.in>

12:20 PM (10 minutes ago)

to beetc1_2024, beetc2_2024, becomp1_2024, becomp2_2024, BEIT_2024, beelect_2024, BE, BE, TPO, Rahul
HSBC :

Kindly finish the registration process on  20th Oct  by 10:00am
Please follow the instructions mentioned below.

1.       Login to career page - https://mycareer.hsbc.com/en_GB/external

2.       Scroll down the page, go to Recently viewed jobs and enter into the text field “Keywords” the correct job number i.e.  0000JGTO and then click Search

3.       Click on “Campus Pipeline Requisition” it takes you to job page then click “Apply”( Pls note this is for the assessment purpose only)

4.       On Privacy statement click “Accept”

5.       On the next page scroll down and click on “New User”

6.       Next page fill up the mandatory fields marked in * and click Register, follow the steps further and complete the registration

 The registration window will be open on 13th Oct , please ensure that they complete the above steps within the stipulated timeline.

 Post that we shall trigger the HSBC Values assessment test on their registered email ID, which is a scenario based test and they have to choose the best and worst possible answers for the scenario provided. It is imperative that they clear HSBC Values assessment before commencement of further rounds of assessment.

 Important test instructions (Mandatory for each applicant to follow):

 ·       Use latest Google Chrome or MS Edge browser only

·       Test should be taken on laptop and not mobile phone

·       Good and uninterrupted internet connectivity for seamless assessment experience

·       Register on career site using same email ID as mentioned in the application list (usage of different email ID will disqualify them from the application process)

·       Any candidate who fails to register on career site during the below stipulated timeline will stand disqualified from further process

·      Kindly complete the assessment before 21st Oct 2023  .
Eligibility: 10th: 65%, 12th: 65%, UG: 6.5 CGPA

Link :  https://mycareer.hsbc.com/en_GB/external/PipelineDetail/Campus-Pipeline-Requisition/205865
Best Of Luck! May you all get the job.
Stay Blessed!"""

In [53]:
loaded_model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)
loaded_model.load_weights('/content/albert_spam_filter.keras')

All PyTorch model weights were used when initializing TFAlbertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFAlbertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [61]:
classifier = pipeline('sentiment-analysis',
                      model=loaded_model,
                      tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)
             )

In [64]:
prediction = classifier(test_email)[0]

In [65]:
print(f"Prediction: {prediction['label']}, confidence: {prediction['score']}")

Prediction: Not Spam, confidence: 0.9925408363342285
